In [84]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
from scipy.stats import pearsonr, spearmanr
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel
from transformers import AutoTokenizer
from tqdm import tqdm
from PipelineCacheWrapper.PipelineCacheWrapper import PipelineCacheWrapper
import spacy
from spacy import displacy
import networkx as nx

In [28]:
from TRExData.LamaTRExData import LamaTRExData
from SentenceTypologyQueryResults import SentenceTypologyQueryResults
from SentenceComparison.SentenceComparison import SentenceComparison
from ModelHelpers.fill_mask_helpers import get_probability_from_pipeline_for_token
from relation_templates.templates import relations, nominalized_relations, get_length_for_relation, get_templates, get_relation_meta, get_relation_name, get_all_templates, relations, get_templates

In [5]:
from TypologyQuerier import TypologyQuerier 

In [6]:
logging.set_verbosity_error()

In [7]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(model(sentence), token)
    return prob

In [30]:
def find_token_index(tree, correct_token):
    for index, token in enumerate(tree):
        if str(token)==correct_token:
            return index
    return -1

In [31]:
def getDependencies(tree, token):
    token_index = find_token_index(tree, SUBJECT_LABEL)
    return list(tree[token_index].subtree)

In [32]:
def getIsDependency(tree, target_token):
    dependencies = list()
    for token in tree:
        for sub_token in list(token.subtree):
            if str(target_token) == str(sub_token):
                dependencies.append(token)
                break
    return dependencies

In [77]:
def tokenDependencyScore(tree, token):
        dependencies = getDependencies(tree, token)
        is_dependencies = getIsDependency(tree, token)
        return (len(dependencies)+len(is_dependencies)) /len(tree)

In [91]:
def getGraph(tree):
    edges = []
    for token in tree:
        for child in token.children:
            edges.append(('{0}'.format(token.lower_),
                          '{0}'.format(child.lower_)))
    return nx.Graph(edges)


def shortestPathLength(tree, src, target):
    graph = getGraph(tree)
    try:
        return nx.shortest_path_length(graph, src, target)
    except:
        return 0
    
    
def spTokenDependencyScore(tree, token):
    return shortestPathLength(tree, SUBJECT_LABEL, OBJECT_LABEL)

In [78]:
def createDependencyTable(relations, token, score_function=tokenDependencyScore):
    relations.sort()
    table = [['relations', 'simple', 'compound', 'complex', 'compound-complex']]
    for relation in relations:
        row = [relation]
        for sentence in get_templates(relation, SUBJECT_LABEL, OBJECT_LABEL).values():
            tree = nlp(sentence)
            score = score_function(tree, token)
            row.append(score)
        table.append(row)
    return table

In [38]:
MASK = "[MASK]"
KEYS = ["active", "passive", "nominalized"]
TOP_K = 1
SUBJECT_LABEL = "subject"
OBJECT_LABEL = "object"

In [9]:
#relations = ["P19", "P36", "P101", "P103","P106","P108", "P178", "P1001"]
#relations = ["P19", "P413", "P159", "P103"]
#relations = ["P364"]
#relations = nominalized_relations

In [10]:
TREx = LamaTRExData(relations = relations)
TREx.load()

In [29]:
nlp = spacy.load("en_core_web_sm")

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
my_tokenizer = lambda sentence: tokenizer(sentence)['input_ids']

In [12]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=TOP_K)

In [13]:
querier = TypologyQuerier(unmasker, relations, TOP_K, MASK)
querier.query(TREx.data)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 37254.71it/s]


In [14]:
unmasker = PipelineCacheWrapper('fill-mask', model='bert-base-cased', top_k=500)

In [15]:
@torch.no_grad()
def metric(sentence: str, token: str):
    prob = get_probability_from_pipeline_for_token(unmasker(sentence), token)
    return prob

In [16]:
Comparer = SentenceComparison(relations, get_templates, metric, MASK, get_relation_meta)
Comparer.compare(TREx.data)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 2583.32it/s]


In [92]:
accuracies = querier.results_for_calculation()
np_accuracies = np.array(accuracies)[1:,1:].flatten().astype(float)

In [93]:
probabilities = Comparer.make_global_comparison()
np_probabilities = np.array(probabilities)[:,1:].flatten().astype(float)

In [94]:
dependency_scores = createDependencyTable(relations, OBJECT_LABEL, spTokenDependencyScore)
np_dependency_scores = np.array(dependency_scores)[1:,1:].flatten().astype(float)

Accuracy vs Length

In [95]:
r_corr , _ = pearsonr(np_accuracies, np_dependency_scores)
r_corr

0.060261873840787766

In [96]:
r_corr , _ = spearmanr(np_accuracies, np_dependency_scores)
r_corr

0.004782275377331492

Probability vs Length

In [89]:
r_corr , _ = pearsonr(np_probabilities, np_dependency_scores)
r_corr

0.20718494328451614

In [90]:
r_corr , _ = spearmanr(np_probabilities, np_dependency_scores)
r_corr

0.24208143210076896